In [1]:
import requests
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras.callbacks import History
from tensorflow.keras.models import save_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout


In [3]:
from utils import *
from dotenv import load_dotenv
import os
load_dotenv()
API_KEY = os.getenv('API_KEY')

In [6]:
def rnn_model(time_step = 100):
  model = Sequential()
  model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
  model.add(Dropout(0.2))
  model.add(LSTM(50, return_sequences=True))
  model.add(Dropout(0.2))
  model.add(LSTM(50))
  model.add(Dropout(0.2))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error', optimizer='adam')

  return model

In [7]:
def train_model(model, X_train, y_train, X_test, y_test , epochs = 100, batch_size = 64):
  history = model.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=1)
  return history

In [10]:

def prepare_and_train_model(symbol, api_key, time_step, epochs, batch_size):
    # Fetch and preprocess data
    df = fetch_stock_data(symbol, api_key)
    data = preprocess_data(df)

    # Scale data
    scaled_data = scale_data(data)

    # Split data into train and test
    train_data, test_data, training_size, test_size = split_data(scaled_data)

    # Create datasets for training/testing
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, y_test = create_dataset(test_data, time_step)

    # Reshape inputs for RNN
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    # Create, train, and save model
    model = rnn_model(time_step)
    history = train_model(model, X_train, y_train, X_test, y_test, epochs, batch_size)
    model.save(f"{symbol}_model.h5")
    print(f"Model saved for {symbol}")
    return history,


In [11]:
def main(symbols, api_key, time_step=100 , epochs=100, batch_size=64 ):
    histories ={}
    for symbol in symbols:
        try:
           print(f"Processing {symbol}")
           history = prepare_and_train_model(symbol, api_key, time_step, epochs, batch_size)
           histories[symbol] = history
        except Exception as e:
            print(f"Error processing {symbol}: {e}")
    return histories

In [ ]:


symbols = [ 'GOOGL', 'AMZN', 'NVDA','MSFT' , 'TSLA']

In [13]:
histories = main(symbols, API_KEY)

Processing GOOGL
Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


65/65 ━━━━━━━━━━━━━━━━━━━━ 17s 164ms/step - loss: 0.0147 - val_loss: 0.0063
Epoch 2/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 20s 165ms/step - loss: 0.0010 - val_loss: 0.0070
Epoch 3/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 11s 174ms/step - loss: 8.9380e-04 - val_loss: 0.0048
Epoch 4/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 20s 164ms/step - loss: 8.6318e-04 - val_loss: 0.0049
Epoch 5/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 20s 162ms/step - loss: 7.6849e-04 - val_loss: 0.0043
Epoch 6/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 21s 174ms/step - loss: 9.0319e-04 - val_loss: 0.0043
Epoch 7/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 20s 174ms/step - loss: 6.9731e-04 - val_loss: 0.0041
Epoch 8/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 19s 148ms/step - loss: 5.6368e-04 - val_loss: 0.0040
Epoch 9/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 11s 160ms/step - loss: 6.4832e-04 - val_loss: 0.0038
Epoch 10/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 21s 174ms/step - loss: 6.1117e-04 - val_loss: 0.0053
Epoch 11/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 20s 174ms/step - loss: 6.9436e-04 - val_loss: 0.0040
Epoch 

Model saved for GOOGL
Processing AMZN
Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

80/80 ━━━━━━━━━━━━━━━━━━━━ 19s 168ms/step - loss: 0.0019 - val_loss: 0.0037
Epoch 2/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 20s 168ms/step - loss: 1.5669e-04 - val_loss: 0.0025
Epoch 3/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 20s 168ms/step - loss: 1.1049e-04 - val_loss: 0.0020
Epoch 4/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 20s 167ms/step - loss: 1.1846e-04 - val_loss: 8.8659e-04
Epoch 5/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 13s 162ms/step - loss: 9.7054e-05 - val_loss: 0.0024
Epoch 6/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 20s 160ms/step - loss: 1.0138e-04 - val_loss: 6.6088e-04
Epoch 7/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 21s 162ms/step - loss: 9.3185e-05 - val_loss: 0.0012
Epoch 8/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 21s 167ms/step - loss: 8.1982e-05 - val_loss: 7.2273e-04
Epoch 9/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 20s 165ms/step - loss: 8.0572e-05 - val_loss: 0.0011
Epoch 10/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 21s 167ms/step - loss: 8.5573e-05 - val_loss: 5.4876e-04
Epoch 11/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 20s 161ms/step - loss: 7.3751e-05 - val

Model saved for MSFT
Processing TSLA
Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 13s 161ms/step - loss: 0.0098 - val_loss: 2.9055e-04
Epoch 2/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 11s 185ms/step - loss: 0.0019 - val_loss: 1.6738e-04
Epoch 3/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 10s 176ms/step - loss: 0.0022 - val_loss: 1.6417e-04
Epoch 4/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 8s 165ms/step - loss: 0.0015 - val_loss: 1.6410e-04
Epoch 5/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 10s 152ms/step - loss: 0.0016 - val_loss: 2.1496e-04
Epoch 6/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 10s 150ms/step - loss: 0.0014 - val_loss: 2.3047e-04
Epoch 7/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 8s 172ms/step - loss: 0.0018 - val_loss: 1.3084e-04
Epoch 8/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 7s 152ms/step - loss: 0.0015 - val_loss: 1.1220e-04
Epoch 9/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 10s 151ms/step - loss: 0.0015 - val_loss: 4.9724e-04
Epoch 10/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 10s 148ms/step - loss: 0.0012 - val_loss: 8.4544e-05
Epoch 11/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 11s 168ms/step - loss: 0.0011 - val_loss: 2.2167e-04
E

Model saved for TSLA


In [14]:
def plot_loss_graphs(histories):
    plt.figure(figsize=(10, 6))

    for symbol, history in histories.items():
        loss = history.history['loss']
        val_loss = history.history.get('val_loss', None)

        plt.plot(loss, label=f'{symbol} Train Loss')
        if val_loss:
            plt.plot(val_loss, linestyle='--', label=f'{symbol} Val Loss')
        plt.title('Training and Validation Loss per Symbol')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()



In [ ]:
plot_loss_graphs(histories)

In [ ]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('MSFT_model.h5')

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = scaler.inverse_transform(y_predicted)
y_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'APPL - Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(y_actual, label='Actual Price')
plt.plot(y_predicted, label='Predicted Price')
plt.title('Appl - Actual vs Predicted')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.grid()
plt.show()

In [ ]:
total_loss = history.history['loss'] + history_2.history['loss']

# Combine validation losses (if used)
if 'val_loss' in history.history:
    total_val_loss = history.history['val_loss'] + history_2.history['val_loss']
else:
    total_val_loss = None

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(total_loss, label='Training Loss')
plt.plot(total_val_loss, label='Validation Loss')
plt.title('Combined Training Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()